In [1]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
df = pd.read_parquet("datasets/df_portfolio_train.parquet")
train_aapl = df[df["tic"] == "AAPL"]

In [3]:
train_aapl

,date,tic,close,vmd_mode_0,vmd_mode_1,vmd_mode_2,high,low,High_VIX,Low_VIX,...,ics,TTM Revenue(Billion),TTM EBITDA(Billion),EBITDA Margin,Price to Book Ratio,PE Ratio,Price to Sales Ratio,Assets,NetIncomeLoss,Return on Equity(%)
0,2011-01-03,AAPL,0.050363,0.055804,-0.011816,-0.004023,0.059087,0.058891,0.210015,0.240301,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
10,2011-01-04,AAPL,0.050626,0.055811,-0.011613,0.003382,0.059488,0.059491,0.213408,0.246270,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
20,2011-01-05,AAPL,0.051040,0.055825,-0.011231,0.004417,0.059817,0.059735,0.210015,0.239591,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
30,2011-01-06,AAPL,0.050999,0.055842,-0.010710,-0.008016,0.059980,0.060352,0.205452,0.238596,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
40,2011-01-07,AAPL,0.051364,0.055862,-0.010063,-0.001357,0.060177,0.060170,0.211419,0.235470,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25110,2020-12-23,AAPL,0.649575,0.663418,0.264630,0.046770,0.663410,0.663858,0.277056,0.314481,...,0.795858,0.696168,0.592054,0.782935,0.587847,0.986924,0.905237,0.0,0.0,-0.346913
25120,2020-12-24,AAPL,0.654585,0.664050,0.287240,-0.110749,0.668570,0.665482,0.267111,0.303965,...,0.795858,0.696168,0.592054,0.782935,0.587847,0.986924,0.905237,0.0,0.0,-0.346913
25130,2020-12-28,AAPL,0.677996,0.664578,0.307290,0.018307,0.688007,0.677716,0.258804,0.300554,...,0.795858,0.696168,0.592054,0.782935,0.587847,0.986924,0.905237,0.0,0.0,-0.346913
25140,2020-12-29,AAPL,0.668969,0.664873,0.323109,0.151125,0.695271,0.681929,0.277524,0.298281,...,0.795858,0.696168,0.592054,0.782935,0.587847,0.986924,0.905237,0.0,0.0,-0.346913


In [8]:
y = train_aapl['close']
X = train_aapl.drop(columns=["date","tic","high","low","close", "vmd_mode_0","vmd_mode_1","vmd_mode_2",])

In [19]:
from sklearn.linear_model import LassoCV

lasso_cv = LassoCV(cv=10).fit(X, y)
best_alpha = lasso_cv.alpha_
print("Best alpha value from cross-validation:", best_alpha)

# Refit Lasso with the best alpha
lasso_best = Lasso(alpha=best_alpha)
lasso_best.fit(X, y)

# Get the coefficients of the features
coefficients = lasso_best.coef_

# Identify the selected features
selected_features = X.columns[coefficients != 0]
print("Selected Features:", selected_features)

Best alpha value from cross-validation: 3.303806884016865e-05
Selected Features: Index(['High_VIX', 'Low_VIX', 'High_SPY', 'Low_SPY', 'Close_SPY', 'Fear Greed',
       'ratingScore', 'ratingDetailsROEScore', 'exports',
       'pound_dollar_exchange_rate', 'unemployment', 'ics',
       'TTM Revenue(Billion)', 'Price to Book Ratio', 'Price to Sales Ratio',
       'Return on Equity(%)'],
      dtype='object')


/Users/ethan/Documents/GitHub/DSE4212/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.909e-03, tolerance: 3.852e-03
  model = cd_fast.enet_coordinate_descent(
/Users/ethan/Documents/GitHub/DSE4212/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.909e-03, tolerance: 3.852e-03
  model = cd_fast.enet_coordinate_descent(
